This notebook is part of the `nbsphinx` documentation: http://nbsphinx.readthedocs.io/.

# Usage

## Sphinx Setup

In the directory with your notebook files, run this command (assuming you have [Sphinx](http://sphinx-doc.org/) installed already):

    python3 -m sphinx.quickstart

Answer the questions that appear on the screen. In case of doubt, just press the `<Return>` key repeatedly to take the default values.

After that, there will be a few brand-new files in the current directory.
You'll have to make a few changes to the file named [conf.py](conf.py). You should at least check if those two variables contain the right things:

```python
extensions = [
    'nbsphinx',
    'sphinx.ext.mathjax',
]
exclude_patterns = ['_build', '**.ipynb_checkpoints']
```

Once your `conf.py` is in place, edit the file named `index.rst` and add the file names of your notebooks (with or without the `.ipynb` extension) to the [toctree](http://www.sphinx-doc.org/en/stable/markup/toctree.html) directive.

<div class="alert alert-info">

**autosummary bug:**

If you are using the `sphinx.ext.autosummary` Sphinx extension, there is [a bug in Sphinx (below version 1.5)](https://github.com/sphinx-doc/sphinx/issues/2485) which prevents notebooks from being parsed. 
As a work-around you can explicitly list all the files for which autosummary should be ran using the  [autosummary_generate](http://www.sphinx-doc.org/en/stable/ext/autosummary.html#confval-autosummary_generate) variable in `conf.py`.  For example,

```python
autosummary_generate = ['myfile1.rst', 'myfile2.rst']
```

</div>

## Running Sphinx

To create the HTML pages, use this command:

    python3 -m sphinx <source-dir> <build-dir>

If you have many notebooks, you can do a parallel build by using the `-j` option:

    python3 -m sphinx <source-dir> <build-dir> -j<number-of-processes>

For example, if your source files are in the current directory and you have 4 CPU cores, you can run this:

    python3 -m sphinx . _build -j4

Afterwards, you can find the main HTML file in `_build/index.html`.

Subsequent builds will be faster, because only those source files which have changed will be re-built.
To force re-building all source files, use the `-E` option.

<div class="alert alert-info">

**Note:**

By default, notebooks will be executed during the Sphinx build process only if they do not have any output cells stored.
See [Controlling Notebook Execution](executing-notebooks.ipynb).

</div>

To create LaTeX output, use:

    python3 -m sphinx <source-dir> <build-dir> -b latex

If you don't know how to create a PDF file from the LaTeX output, you should have a look at [Latexmk](http://personal.psu.edu/jcc8//software/latexmk-jcc/) (see also [this tutorial](http://mg.readthedocs.io/latexmk.html)).

Sphinx can automatically check if the links you are using are still valid.
Just invoke it like this:

    python3 -m sphinx <source-dir> <build-dir> -b linkcheck

## Watching for Changes with `sphinx-autobuild`

If you think it's tedious to run the Sphinx build command again and again while you make changes to your notebooks, you'll be happy to hear that there is a way to avoid that: [sphinx-autobuild](https://pypi.python.org/pypi/sphinx-autobuild)!

It can be installed with

    python3 -m pip install sphinx-autobuild --user

You can start auto-building your files with

    python3 -m sphinx_autobuild <source-dir> <build-dir>

This will start a local webserver which will serve the generated HTML pages at http://localhost:8000/.
Whenever you save changes in one of your notebooks, the appropriate HTML page(s) will be re-built and when finished, your browser view will be refreshed automagically.
Neat!

You can also abuse this to auto-build the LaTeX output:

    python3 -m sphinx_autobuild <source-dir> <build-dir> -b latex

However, to auto-build the final PDF file as well, you'll need an additional tool.
Again, you can use `latexmk` for this (see [above](#Running-Sphinx)).
Change to the build directory and run

    latexmk -pdf -pvc

If your PDF viewer isn't opened because of LaTeX build errors, you can use the command line flag `-f` to *force* creating a PDF file.

## Automatic Creation of HTML and PDF output on [readthedocs.org](https://readthedocs.org)

There are two different methods, both of which are described below.

In both cases, you'll first have to create an account on https://readthedocs.org/ and connect your Github/Bitbucket account. Instead of connecting, you can also manually add any publicly available Git/Subversion/Mercurial/Bazaar repository.

After doing the steps described below, you only have to "push" to your repository, and the HTML pages and the PDF file of your stuff are automagically created on readthedocs.org. Awesome!

You can even have different versions of your stuff, just use Git tags and branches and select in the readthedocs.org settings (under "Admin", "Versions") which of those should be created.

If your new versions are not automatically built, go to the “Settings” of your Github repository, continue to “Integrations & services”, and make sure that “ReadTheDocs” is listed and activated in the “Services” section. If not, use “Add service”. There is probably a similar thing for Bitbucket and others.

<div class="alert alert-info">

**Note:**

If you want to execute notebooks (see [Controlling Notebook Execution](executing-notebooks.ipynb)), you'll need to install the appropriate Jupyter kernel.  In the examples below, the IPython kernel ist installed from the packet `ipykernel`.

</div>

### Using `requirements.txt`

1. Create a file named `requirements.txt` (or whatever name you wish) in your repository containing the required pip packages:

        sphinx>=1.4
        ipykernel
        nbsphinx
        
    You can also install directly from Github et al., using a specific branch/tag/commit, e.g.

        git+https://github.com/spatialaudio/nbsphinx.git@master

1. In the "Advanced Settings" on readthedocs.org, specify the path to your `requirements.txt` file (or however you called it) in the box labeled "Requirements file". Kinda obvious, isn't it?

1. Still in the "Advanced Settings", make sure the right Python interpreter is chosen.  This must be the same version (2.x or 3.x) as you were using in your notebooks!

### Using `conda`

1. Create a file named `readthedocs.yml` in the main directory of your repository, containing the name of yet another file:

        conda:
          file: readthedocs-environment.yml

1. Create the file mentioned above.
You can choose whatever name you want (it may also live in a subdirectory, e.g. `doc/environment.yml`), it just has to match whatever is specified in `readthedocs.yml`.
The second file describes a [conda environment](https://conda.io/docs/using/envs.html) and should contain something like this:

        channels:
          - conda-forge
        dependencies:
          - python==3.5
          - sphinx>=1.4
          - pandoc
          - nbconvert
          - ipykernel
          - pip:
            - nbsphinx

    It is up to you if you want to install `nbsphinx` with `conda` or with `pip` (but note that the `conda` package might be outdated).
    And you can of course add further `conda` and `pip` packages. You can also install packages directly from Github et al., using a specific branch/tag/commit, e.g.
    
          - pip:
            - git+https://github.com/spatialaudio/nbsphinx.git@master

<div class="alert alert-info">

**Note:**

The specification of the `conda-forge` channel is recommended because it tends to have more recent package versions than the default channel.

</div>

<div class="alert alert-info">

**Note:**

Most of the "Advanced Settings" on readthedocs.org will be ignored if you have a `readthedocs.yml` file.

</div>

<div class="alert alert-warning">

**Warning:**

If you have a very long repository name (or branch name), you might run into this quite obscure problem: ['placeholder too short'](https://github.com/rtfd/readthedocs.org/issues/1902).

</div>

## HTML Themes

The `nbsphinx` extension does *not* provide its own theme, you can use any of the available themes or [create a custom one](http://www.sphinx-doc.org/en/stable/theming.html#creating-themes), if you feel like it.

The following (incomplete) list of themes contains up to three links for each theme:

1. The documentation (or the official sample page) of this theme (if available; see also the [documentation of the built-in Sphinx themes](http://www.sphinx-doc.org/en/latest/theming.html#builtin-themes))
1. How the `nbsphinx` documentation looks when using this theme
1. How to enable this theme using either `requirements.txt` or `readthedocs.yml` and theme-specific settings (in some cases)

### Sphinx's Built-In Themes

* [alabaster](https://alabaster.readthedocs.io/):
  [example](http://nbsphinx.readthedocs.io/en/alabaster-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/alabaster-theme^...alabaster-theme)

* `pyramid`:
  [example](http://nbsphinx.readthedocs.io/en/pyramid-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/pyramid-theme^...pyramid-theme)

* `classic`:
  [example](http://nbsphinx.readthedocs.io/en/classic-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/classic-theme^...classic-theme)

* `bizstyle`:
  [example](http://nbsphinx.readthedocs.io/en/bizstyle-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/bizstyle-theme^...bizstyle-theme)

* `haiku`:
  [example](http://nbsphinx.readthedocs.io/en/haiku-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/haiku-theme^...haiku-theme)

* `traditional`:
  [example](http://nbsphinx.readthedocs.io/en/traditional-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/traditional-theme^...traditional-theme)

* `agogo`:
  [example](http://nbsphinx.readthedocs.io/en/agogo-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/agogo-theme^...agogo-theme)

* `nature`:
  [example](http://nbsphinx.readthedocs.io/en/nature-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/nature-theme^...nature-theme)

### 3rd-Party Themes

* [sphinx_rtd_theme](https://github.com/rtfd/sphinx_rtd_theme):
  [example](http://nbsphinx.readthedocs.io/en/rtd-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/rtd-theme^...rtd-theme)

* [bootstrap](http://sphinx-bootstrap-theme.readthedocs.io/):
  [example](http://nbsphinx.readthedocs.io/en/bootstrap-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/bootstrap-theme^...bootstrap-theme)

* [cloud](https://pythonhosted.org/cloud_sptheme/):
  [example](http://nbsphinx.readthedocs.io/en/cloud-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/cloud-theme^...cloud-theme)

* [sphinx_py3doc_enhanced_theme](https://github.com/ionelmc/sphinx-py3doc-enhanced-theme):
  [example](http://nbsphinx.readthedocs.io/en/py3doc-enh-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/py3doc-enh-theme^...py3doc-enh-theme)

* [basicstrap](http://pythonhosted.org/sphinxjp.themes.basicstrap/):
  [example](http://nbsphinx.readthedocs.io/en/basicstrap-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/basicstrap-theme^...basicstrap-theme)

* [dotted](https://pythonhosted.org/sphinxjp.themes.dotted/):
  [example](http://nbsphinx.readthedocs.io/en/dotted-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/dotted-theme^...dotted-theme)

* [better](https://sphinx-better-theme.readthedocs.io/):
  [example](http://nbsphinx.readthedocs.io/en/better-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/better-theme^...better-theme)

* [guzzle_sphinx_theme](https://github.com/guzzle/guzzle_sphinx_theme):
  [example](http://nbsphinx.readthedocs.io/en/guzzle-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/guzzle-theme^...guzzle-theme)

* [julia](https://github.com/JuliaLang/JuliaDoc):
  [example](http://nbsphinx.readthedocs.io/en/julia-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/julia-theme^...julia-theme)

* [jupyter](https://github.com/jupyter/jupyter-sphinx-theme/):
  [example](http://nbsphinx.readthedocs.io/en/jupyter-theme/),
  [usage](https://github.com/spatialaudio/nbsphinx/compare/jupyter-theme^...jupyter-theme)

If you know of another Sphinx theme that should be included here, please open an [issue on Github](https://github.com/spatialaudio/nbsphinx/issues).

## Using Notebooks with Git

[Git](https://git-scm.com/) is extremely useful for managing source code and it can and should also be used for managing Jupyter notebooks.
There is one caveat, however:
Notebooks can contain output cells with rich media like images, plots, sounds, HTML, JavaScript and many other types of bulky machine-created content.
This can make it hard to work with Git efficiently, because changes in those bulky contents can completely obscure the more interesting human-made changes in text and source code.
Working with multiple collaborators on a notebook can become very tedious because of this.

It is therefore highly recommended that you remove all outputs from your notebooks before committing changes to a Git repository (except for the reasons mentioned in [Pre-Executing Notebooks](pre-executed.ipynb)).

If there are no output cells in a notebook, `nbsphinx` will by default execute the notebook, and the pages generated by Sphinx will therefore contain all the output cells.
See [Controlling Notebook Execution](executing-notebooks.ipynb) for how this behavior can be customized.

In the Jupyter Notebook application, you can manually clear all outputs by selecting
"Cell" $\to$ "All Output" $\to$ "Clear" from the menu.

There are several tools available to remove outputs from multiple files at once without having to open them separately.
You can even include such a tool as "clean/smudge filters" into your Git workflow, which will strip the output cells automatically whenever a Git command is executed.
For details, have a look at those links:

* https://github.com/kynan/nbstripout
* https://github.com/toobaz/ipynb_output_filter
* http://tillahoffmann.github.io/2017/04/17/versioning-jupyter-notebooks-with-git.html
* http://timstaley.co.uk/posts/making-git-and-jupyter-notebooks-play-nice/
* http://pascalbugnion.net/blog/ipython-notebooks-and-git.html

